In [ ]:
# #############################################################################
# Importing necessary libraries for creating model
from keras.models import model_from_json
from keras.models import load_model
import os
import pandas as pd
import numpy as np
import keras
import cv2 # importing open CV library
import dlib   # For frontal face identity
from PIL import Image
import csv
import imutils
import face_recognition # importing face recognition library
from sklearn.metrics import classification_report, confusion_matrix
import sys
from zipfile import ZipFile
import tarfile
from urllib.request import urlopen
from scipy.io import loadmat

## 1- Data extraction

In [ ]:
#variable to be defined
dataset='Gender_Shade'
data_path='../../Data/PPB-2017.zip'
output_path= '../../Data/Gender_Shade/'

In [ ]:
#### Data extraction:
def extract_data(dataset, data_path, output_path):
    if dataset=='imdb':
        tar = tarfile.open(data_path,"r:")
        tar.extractall(output_path)
        tar.close()
    else:
        with ZipFile(data_path, 'r') as zipObj:
        # Extract all the contents of zip file 
            zipObj.extractall(output_path) 

In [ ]:
# If data is scholar, use this code to extract image_path:
extract_data(dataset, data_path, output_path)
images=[]
def replace_func(text):
    chars = ['[',']','u',"'"]
    for c in chars:
        #print(c)
        text = text.replace(c, '')
    return text
#scholar
def scholar(file_path,gender):
    scholar=pd.read_csv(file_path,header=None)
    scholar=scholar.drop(0,axis=1)
    Scholar_with_gender= pd.DataFrame()
    images_path=[]
    for (columnName, columnData) in scholar.iteritems():
        images_path.extend(columnData.values)
    Scholar_with_gender['Image_name']=images_path
    Scholar_with_gender['true_gender']=gender
    return Scholar_with_gender

scholar_female=scholar(output_path+'female_name_url.csv','f')
scholar_male=scholar(output_path+'male_name_url.csv','m')
scholar_all = pd.concat([scholar_female,scholar_male],ignore_index=True)
scholar_all['Image_name']= scholar_all['Image_name'].apply(lambda x: replace_func(str(x)))
images= scholar_all['Image_name']
print('Total number of images:', len(images))

In [ ]:
#if the data is one of 5 remaining, use this code to extract image_path
extract_data(dataset, data_path, output_path)
images=[]
if len(next(os.walk(output_path))[1]) == 0: #case when images are in one folder
        for image in os.listdir(output_path): 
            if image[-3:] == 'jpg' or image[-3:] == 'png':
                images.append(output_path + image)
else: #case when images are in several folders
        for path in [output_path + next(os.walk(output_path))[1][n] for n in range(len(next(os.walk(output_path))[1]))]:
            if len(next(os.walk(path))[1]) == 0:
                for image in os.listdir(path+'/'): 
                    if image[-3:] == 'jpg' or image[-3:] == 'png':
                           images.append(path+'/' + image)
            else:
                for sub_path in [path+'/' + next(os.walk(path))[1][n] for n in range(len(next(os.walk(path))[1]))]:
                    for image in os.listdir(sub_path+'/'): 
                        if image[-3:] == 'jpg' or image[-3:] == 'png':
                               images.append(path+'/' + image)
print('Total number of images:', len(images))

## 2- Load and run the model 

In [ ]:
# if the dataset is scholar used this function to read the images
def url_to_image(url):
# download the image, convert it to a NumPy array, and then read
# it into OpenCV format
    resp = urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
# return the image
    return image


In [ ]:

# #############################################################################
# loading the trained model and weights
json_file = open('../trained model/model.json', 'r')     # Loading the jason file
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("../trained model/model.h5")    # Loading the weights


def detect_faces(image):                    # Create a face detector
    face_detector = dlib.get_frontal_face_detector()
    detected_faces = face_detector(image, 1)    # Run detector and get bounding boxes of the faces on image
    face_frames = [(x.left(), x.top(),
                    x.right(), x.bottom()) for x in detected_faces]  # To identify the frame size
    return face_frames


def resize_im(image, hight, width):          # Resizing the image
    resized_im = image.resize((hight, width))
    return resized_im


def reshape(numpy_array):            # Reshape the image
    d = np.stack(numpy_array)
    reshaped = d.reshape(1, d.size)
    return reshaped


def CNN_baseline(image_path):
    output = pd.DataFrame()
    img= cv2.imread(image_path)
    #img= url_to_image(image_path)     #uncomment this if your dataset is scholar
    detected_faces = detect_faces(img)
    for n, face_rect in enumerate(detected_faces):
                    fa = Image.fromarray(img).crop(face_rect)   # Cropping the face or faces
                    s = np.array(resize_im(fa, 100,100))
                    c = cv2.cvtColor(s, cv2.COLOR_BGR2GRAY)    # Converting the image
                    re_data = reshape(c)
                    # Save the converted image data
                    dic = { i : re_data[0][i] for i in range(0, len(re_data[0]) ) }
                    output = output.append(dic, ignore_index=True)
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()    # Need to kill the image window
    df_x= output.iloc[:,0:].values.reshape(len(output),100,100,1)
    f_x = np.array(df_x)  # Need to convert as a array
    loaded_model.compile(loss='categorical_crossentropy',optimizer = 'adadelta', metrics = ['accuracy'])
    prob = np.asmatrix(loaded_model.predict(f_x))
    if prob[0,0] > prob[0,1]:
             gender= 'f'
    if prob[0,0] < prob[0,1]:
             gender='m'
    
    return gender

In [ ]:
with open(output_path+dataset+'.csv', 'w', newline='') as myfile:             # Here output written in the file
        wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
        wr.writerow(['Image_name','Pred_Gender'])
        for imagelocation in images:                        # Get the images location
                try:
                    CNN_baseline_gender=CNN_baseline(imagelocation)   # return the predicted gender
                    wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
                    wr.writerow([imagelocation,CNN_baseline_gender])
                except:
                    wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
                    wr.writerow([imagelocation,'u'])   #if there is an error in returning gender , write undefined
                    print("Oops!",sys.exc_info()[0],"occured.")
                    #print()

## 3- Extract the annotations

In [ ]:
## for OUI Dataset
def oui_annotation_extraction(annotation_path,annotation_output_path):
    extract_data(dataset, annotation_path, annotation_output_path)  #unzip the file 
    files=[annotation_output_path+'/'+name for name in os.listdir(annotation_output_path) if os.path.isfile(os.path.join(annotation_output_path, name))]
    df = pd.DataFrame()
    for file in files:
        data = pd.read_csv(file, sep="\t")  #load each file as dataframe
        df= df.append(data, ignore_index = True)
    df= df.astype(str)
    df['Image_name']=df['user_id']+'/'+ df['face_id']+'.'+df['original_image'] # make Image name similar to the one in the image path
    df= df[['Image_name','gender']] # keep only gender and imagename
    df=df.rename(columns={"gender": "true_gender"})
    df['true_gender']=df['true_gender'].apply(lambda x: 'u' if x=='nan' else x)
    return df.drop_duplicates()

In [ ]:
oui_annotation= oui_annotation_extraction('../../Data/OUI_annotations.zip','../../Data/OUI_annotations')
print(oui_annotation.head(2),'\n')
print('the shape of the annotation dataframe is',oui_annotation.shape)

In [ ]:
## For Gender_shade dataset
shade_annotation= pd.read_csv('../../Data/PPB-2017-metadata.csv')
shade_annotation= shade_annotation[['filename','gender']]
shade_annotation.columns = ['Image_name', 'true_gender']
shade_annotation['true_gender']= shade_annotation['true_gender'].apply(lambda x: 'f' if x=='Female' else 'm')
print(shade_annotation.head(2),'\n')
print('the shape of the annotation dataframe is',shade_annotation.shape)

In [ ]:
## For IMDB and wiki datasets
############################
### for .mat file for IMDB ###
def get_names(x):
    if len(x)>0:
        return x[0]
    else:
        return ''
dataset ='imdb'
### for .mat file for imdb and wiki ###
if dataset == 'imdb' or dataset == 'wiki':
    path_to_meta = path_to_output + dataset + ".mat"
    mat = loadmat(path_to_meta)  # load mat-file
    mdata = mat[dataset]  # variable in mat file
    mdtype = mdata.dtype
    ndata = {n: mdata[n][0, 0] for n in mdtype.names}
    columns = [n for n, v in ndata.items()]# if v.size == ndata['numIntervals']] 
    full_path = [mdata['full_path'][0,0][0][n][0] for n in range(len(mdata['full_path'][0,0][0]))]
    gender = mdata['gender'][0,0][0]
    if dataset == 'imdb':
        imdb_annotation = pd.DataFrame({ "Image_name":full_path, "true_gender":gender})
    elif dataset=='wiki':
        wiki_annotation = pd.DataFrame({ "Image_name":full_path, "true_gender":gender})

In [ ]:
###For twitter dataset

metadf = pd.DataFrame()
meta_path = '/Users/nadabeili/Desktop/twitter/_a_results32langs/'
for file in os.listdir(meta_path):
    df = pd.read_csv(meta_path + file)
    metadf = metadf.append(df)
    
metadf.reset_index(inplace = True)
metadf

## 4- Evaluation

In [ ]:
### Precision, recall, F1_score
#read the result_file
df_pred=pd.read_csv(output_path+dataset+'.csv')
## make image names similar to the annotation
df_pred['Image_name']= df_pred['Image_name'].apply(lambda x: x.split('/')[-1])
##
result_data= pd.merge(shade_annotation,df_pred, how= 'inner', on='Image_name')
result_data.to_csv('../../Data/all_result_Gender_shade.csv')
result_data= result_data[result_data.Pred_Gender !='u']
# True values
y_true = result_data['true_gender']
# Predicted values
y_pred = result_data['Pred_Gender']

# Print the confusion matrix
print(confusion_matrix(y_true, y_pred))
# Print the precision and recall, among other metrics
print(classification_report(y_true, y_pred, digits=4))

#printing the metrics
metrics_dict=classification_report(y_true, y_pred,output_dict=True)

#precision:
print('Precision:',round(metrics_dict['weighted avg']['precision'],4))
#Recall
print('Recall:',round(metrics_dict['weighted avg']['recall'],4))
#F1-score
print('F1-score:',round(metrics_dict['weighted avg']['f1-score'],4))
#accuracy
print('Accuracy:',round(metrics_dict['accuracy'],4))

In [ ]:
result_data